Import data in PNG form and split into Train, Test, Validate and load


In [38]:
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split
from torchvision.transforms import ToTensor, Normalize, Compose, Grayscale

train_set = '../Data/mnist-png-format/train'
test_set = '../Data/mnist-png-format/test'
 
batch_size = 32
workers = 8

transforms = Compose([
                        Grayscale(num_output_channels=1),   # PNG file is RGB 3 layer convert to greyscal 1 layer
                        ToTensor(),           
                        Normalize(mean=(0.5), std=(0.5)), 
                        ])

train_dataset = ImageFolder(train_set, transform=transforms)
test_dataset = ImageFolder(test_set, transform=transforms)

# Split Train dataset set into two (Train=85/Validation=15% split)
train_size = int(len(train_dataset) * 0.85) 
validation_size = (len(train_dataset) - train_size) 
train, validation = random_split(train_dataset, [train_size, validation_size])

# load data into usable format, mix/shuffle data so data is not in order 
train_data = DataLoader(train, batch_size = batch_size, shuffle = True, num_workers = workers)
val_data  = DataLoader(validation, batch_size = batch_size, shuffle = True, num_workers = workers)
test_data  = DataLoader(test_dataset, batch_size = batch_size, shuffle = True, num_workers = workers)

In [39]:
from torch import nn, optim
import torch.nn.functional as F
import torch

class SingleLayerModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden_1 = nn.Linear(28*28,100) # 28x28 Input image, number of neurons
        self.output = nn.Linear(100,10) # 10 class output 0-9


    def forward(self,x):
        x = x.view(x.shape[0], -1) 
        x = F.relu(self.hidden_1(x))
        y = self.output(x)
        return y

In [40]:
ModelMLP  = SingleLayerModel()
optimiser = optim.Adam(ModelMLP.parameters())
loss_func = nn.CrossEntropyLoss()
train_epochs = 5

In [37]:

  for i in range(train_epochs): # training epochs
    epoch_acc = 0

    for (imgs, labels) in train_data: # loop through each batch  
      pred_y = ModelMLP(imgs)
      loss = loss_func(pred_y, labels)

      optimiser.zero_grad()
      loss.backward()
      optimiser.step()

      pred = torch.max(pred_y, 1)[1]
      correct_batch = (pred == labels).sum().item() # Calculate correct prediction where prediction == label 

      print(correct_batch/batch_size + '%')
    epoch_acc += correct_batch
    print(epoch_acc/(i+1*batch_size))


0.71875
1.3636363636363635


KeyboardInterrupt: 